In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Flatten, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import pydot_ng
import graphviz
import os

In [2]:
num_classes = 7
batch_size = 64
epochs = 100
Img_height = 48
Img_width = 48

In [3]:
data = pd.read_csv("Desktop\emotion.csv")
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
pixels = data['pixels'].tolist() 
faces = []

for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(Img_height, Img_width) 
    faces.append(face.astype('float32'))

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

emotions = pd.get_dummies(data['emotion']).values

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

Emotions Model

In [6]:
model = Sequential()

# Block-1: The First Convolutional Block

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal',
                 activation="relu", 
                 input_shape=(Img_height, Img_width, 1), 
                 name="Conv1"))

model.add(BatchNormalization(name="Batch_Norm1"))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal', 
                 activation="relu", name="Conv2"))

model.add(BatchNormalization(name="Batch_Norm2"))
model.add(MaxPooling2D(pool_size=(2,2), name="Maxpool1"))
model.add(Dropout(0.5, name="Dropout1"))

# Block-2: The Second Convolutional Block

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal',
                 activation="relu", name="Conv3"))

model.add(BatchNormalization(name="Batch_Norm3"))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',
                 kernel_initializer='he_normal', 
                 activation="relu", name="Conv4"))

model.add(BatchNormalization(name="Batch_Norm4"))
model.add(MaxPooling2D(pool_size=(2,2), name="Maxpool2"))
model.add(Dropout(0.5, name="Dropout2"))

# Block-3: The Third Convolutional Block

model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal', 
                 activation="relu", name="Conv5"))

model.add(BatchNormalization(name="Batch_Norm5"))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal',
                 activation="relu", name="Conv6"))

model.add(BatchNormalization(name="Batch_Norm6"))
model.add(MaxPooling2D(pool_size=(2,2), name="Maxpool3"))
model.add(Dropout(0.5, name="Dropout3"))

# Block-4: The Fully Connected Block

model.add(Flatten(name="Flatten"))
model.add(Dense(64, activation="relu", kernel_initializer='he_normal', name="Dense"))
model.add(BatchNormalization(name="Batch_Norm7"))
model.add(Dropout(0.5, name="Dropout4"))

# Block-5: The Output Block

model.add(Dense(num_classes, activation="softmax", kernel_initializer='he_normal', name = "Output"))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 48, 48, 32)        320       
                                                                 
 Batch_Norm1 (BatchNormaliza  (None, 48, 48, 32)       128       
 tion)                                                           
                                                                 
 Conv2 (Conv2D)              (None, 48, 48, 32)        9248      
                                                                 
 Batch_Norm2 (BatchNormaliza  (None, 48, 48, 32)       128       
 tion)                                                           
                                                                 
 Maxpool1 (MaxPooling2D)     (None, 24, 24, 32)        0         
                                                                 
 Dropout1 (Dropout)          (None, 24, 24, 32)        0

In [8]:
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

C:\Users\ayush\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

tensorboard = TensorBoard(log_dir='logs3')
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint("emotions3.h5", monitor='val_accuracy', verbose=1, save_best_only=True)

In [ ]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])

Epoch 1/100
455/455 [==============================] - ETA: 0s - loss: 2.0569 - accuracy: 0.2566
Epoch 1: val_accuracy improved from -inf to 0.37113, saving model to emotions3.h5
455/455 [==============================] - 336s 734ms/step - loss: 2.0569 - accuracy: 0.2566 - val_loss: 1.6223 - val_accuracy: 0.3711 - lr: 0.0010
Epoch 2/100
455/455 [==============================] - ETA: 0s - loss: 1.6320 - accuracy: 0.3779
Epoch 2: val_accuracy improved from 0.37113 to 0.43410, saving model to emotions3.h5
455/455 [==============================] - 340s 747ms/step - loss: 1.6320 - accuracy: 0.3779 - val_loss: 1.4626 - val_accuracy: 0.4341 - lr: 0.0010
Epoch 3/100
455/455 [==============================] - ETA: 0s - loss: 1.4934 - accuracy: 0.4232
Epoch 3: val_accuracy improved from 0.43410 to 0.47896, saving model to emotions3.h5
455/455 [==============================] - 436s 960ms/step - loss: 1.4934 - accuracy: 0.4232 - val_loss: 1.3372 - val_accuracy: 0.4790 - lr: 0.0010
Epoch 4/100
4